In [58]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [6]:
import winsound
duration = 1050  # milliseconds
freq = 940  # Hz
winsound.Beep(freq, duration)

In [78]:
sentences = ['Editorial: Counting down to Election Day; learn and vote', 
          'Donald Trump Rips Mike Pence for Not Having Courage on 1/6 to Reject Election Results', 
          'Mark Larson on the recall election’s latest',
          'Personal threats, election lies and punishing new laws rattle election officials, raising fears of a mass exodus',
          'Nicaragua: Another presidential contender arrested ahead of election',
          'Tiburon opts to hold special election to fill vacant council seat',
          'PM Imran Khans party bags majority in Pakistans Kashmir election',
          'DISTRICT 7: Special election between Sonya McKinstry, Cassius Lanier is Tuesday',
          'Arizona GOP Lawmaker Booed Off Stage at Trump Protect Our Elections Rally',
          'The Path Toward Commonsense Election Reform',
          'Merkel’s Bloc Continues to Slide in German Election Poll',
          'Housing, homelessness at forefront as Seattle voters weigh primary election choices',
          'Marin scrambles to secure polling places for Newsom recall election',
          'Somalia postpones long-delayed election',
          'Trumps impact on the line in Texas special election',
          'AP FACT CHECK: Trump is relentless in election fabrications',
          'Letter: Support Harless in election',
          'Former AG Woods Criticizes Maricopa County Election Audit, Sen. Sinema',
          'More than 700 votes cast in early voting for mayoral, Topeka City Council District 3 and Seaman BOE primaries',
          'League reminds residents of special election for Select Board',
          'Election Commission votes to stay in the courthouse'
         ]
    
    
sources = ['The Daily Courier', 
           'Newsweek',
           'Kusi',
           'CNN',
           'DW',
           'mij',
           'reuters',
           'The Tuscaloosa News',
           'Newsweek',
           'The Ripon Forum',
           'Bloomberg',
           'The Seattle Times',
           'The Mercury Times',
           'France 24',
           'Politico',
           'KSNBlocal',
           'The Columbian',
           'KJZZ',
           'cjonline',
           'WickedLine.com',
           'Johnson City Press'
          ]

In [79]:
#sentences = ['This framework generates embeddings for each input sentence',
#    'Sentences are passed as a list of string.', 
#    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [85]:
sentence_df = pd.DataFrame()
sentence_arr = []
embedding_arr = []

i = 0

for sentence, embedding in zip(sentences, sentence_embeddings):
    #print("Sentence:", sentence)
    #print("Embedding:", embedding)
    #print("")    
    sentence_arr.append(sentences[i])
    embedding_arr.append(embedding)
    i = i + 1

sentence_df['text'] = sentence_arr
sentence_df['emb'] = embedding_arr
sentence_df['sources'] = sources
print(sentence_df)

                                                 text  \
0   Editorial: Counting down to Election Day; lear...   
1   Donald Trump Rips Mike Pence for Not Having Co...   
2         Mark Larson on the recall election’s latest   
3   Personal threats, election lies and punishing ...   
4   Nicaragua: Another presidential contender arre...   
5   Tiburon opts to hold special election to fill ...   
6   PM Imran Khans party bags majority in Pakistan...   
7   DISTRICT 7: Special election between Sonya McK...   
8   Arizona GOP Lawmaker Booed Off Stage at Trump ...   
9         The Path Toward Commonsense Election Reform   
10  Merkel’s Bloc Continues to Slide in German Ele...   
11  Housing, homelessness at forefront as Seattle ...   
12  Marin scrambles to secure polling places for N...   
13            Somalia postpones long-delayed election   
14  Trumps impact on the line in Texas special ele...   
15  AP FACT CHECK: Trump is relentless in election...   
16                Letter: Suppo

In [92]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 5):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
#sentence_df['distance_from_centroid'] = sentence_df.apply(distance_from_centroid, axis=1)

clustering_question(sentence_df)

(                                                 text  \
 0   Editorial: Counting down to Election Day; lear...   
 1   Donald Trump Rips Mike Pence for Not Having Co...   
 2         Mark Larson on the recall election’s latest   
 3   Personal threats, election lies and punishing ...   
 4   Nicaragua: Another presidential contender arre...   
 5   Tiburon opts to hold special election to fill ...   
 6   PM Imran Khans party bags majority in Pakistan...   
 7   DISTRICT 7: Special election between Sonya McK...   
 8   Arizona GOP Lawmaker Booed Off Stage at Trump ...   
 9         The Path Toward Commonsense Election Reform   
 10  Merkel’s Bloc Continues to Slide in German Ele...   
 11  Housing, homelessness at forefront as Seattle ...   
 12  Marin scrambles to secure polling places for N...   
 13            Somalia postpones long-delayed election   
 14  Trumps impact on the line in Texas special ele...   
 15  AP FACT CHECK: Trump is relentless in election...   
 16           

In [96]:
filter = sentence_df.cluster.eq(0)
print(sentence_df[filter])

                                                 text  \
1   Donald Trump Rips Mike Pence for Not Having Co...   
14  Trumps impact on the line in Texas special ele...   
15  AP FACT CHECK: Trump is relentless in election...   

                                                  emb    sources  cluster  \
1   [-0.6118304, 0.61012614, 0.17613193, 0.3042496...   Newsweek        0   
14  [-0.3577849, -0.37728858, 0.5586421, 0.4288315...   Politico        0   
15  [-0.07392996, 1.0663792, 0.5958656, 0.35001138...  KSNBlocal        0   

                                             centroid  
1   [-0.34778154, 0.43322685, 0.44358373, 0.361028...  
14  [-0.34778154, 0.43322685, 0.44358373, 0.361028...  
15  [-0.34778154, 0.43322685, 0.44358373, 0.361028...  


In [97]:
filter = sentence_df.cluster.eq(1)
print(sentence_df[filter])

                                                 text  \
3   Personal threats, election lies and punishing ...   
4   Nicaragua: Another presidential contender arre...   
8   Arizona GOP Lawmaker Booed Off Stage at Trump ...   
11  Housing, homelessness at forefront as Seattle ...   

                                                  emb            sources  \
3   [0.0055751894, 0.52150637, 0.65236294, 0.03890...                CNN   
4   [-0.6060503, -0.18584813, 0.5675973, 0.4749131...                 DW   
8   [-0.08417319, 1.0602788, -0.18716964, 0.128213...           Newsweek   
11  [0.25419423, 0.9853946, 0.8054611, 0.074949816...  The Seattle Times   

    cluster                                           centroid  
3         1  [-0.10760627, 0.59532815, 0.45957527, 0.179235...  
4         1  [-0.10760627, 0.59532815, 0.45957527, 0.179235...  
8         1  [-0.10760627, 0.59532815, 0.45957527, 0.179235...  
11        1  [-0.10760627, 0.59532815, 0.45957527, 0.179235...  


In [48]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 2):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)

clustering_question(sentence_df)

NameError: name 'data' is not defined

In [ ]:
def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
sentence_df['distance_from_centroid'] = sentence_df.apply(distance_from_centroid, axis=1)



In [7]:
! python -m pip install --upgrade pip --quiet
! pip install spacy --quiet
! pip3 install spacy-transformers --quiet
! python -m spacy download en_trf_bertbaseuncased_lg --quiet



[x] No compatible package found for 'en_trf_bertbaseuncased_lg' (spaCy v3.1.1)



In [10]:
#Load bert model
nlp = en_trf_bertbaseuncased_lg.load()

# Utility function for generating sentence embedding from the text
def get_embeddinngs(text):
    return nlp(text).vector

# Generating sentence embedding from the text
data['emb'] = data['text'].apply(get_embeddinngs)

NameError: name 'data' is not defined

In [6]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 2):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)



In [ ]:
#K-means

kmeans = KMeans(n_clusters=2).fit(deltadf)

sklearn.cluster.k_means
